# Fairness in complex network analysis


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from collections import Counter
import igraph as ig
import partition_igraph
import random 
from abcd_graph import ABCDGraph, ABCDParams
import statsmodels.api as sm
from statsmodels.genmod import families
import json


# (1) Web developers graph

We look at the Web developers graph already introduced in Chapter 1.
Recall that there are two classes of developers: machine learning (ml) and web.

Variables:
* **Y** = "is a ml developer", the target variable 
* **D_web** = number of neighbours that are web developers ("web degree")
* **D_ml** = number of neighbours that are ml developers ("ml degree")
* **X** = Protected variable (binary) ; we pick one the the supplied binary features from ther dataset ; this feature is correlated with Y.

For the prediction models, we use logistic regression and pick a decision threshold $\tau$ to maximize accuracy. 

The protected variable is significantly correlated with the target, so we expect the model to be unfair.
It is also correlated with the degree-based features, so fairness by unawareness (removing X) will likely not be enough.

We demonstrate the existence of unfairness w.r.t. X when predicting Y via the two measures "DI" and "SP" detailed in the textbook.

The unfairness can be lowered using two different decision thresholds depending on the value of X, $\tau_0$ and $\tau_1$.


In [ ]:
## setting the path to the datasets
datadir = "../Datasets/"

In [ ]:
## read the GitHub developers edge list as tuples and build undirected graph
## each node index is stored in vertex attribute "id"
df = pd.read_csv(datadir + "GitHubDevelopers/musae_git_edges.csv")
G = ig.Graph.TupleList(
    [tuple(x) for x in df.values], directed=False, vertex_name_attr="id"
)

## read node class (ml or web) and save: Y=1 for ml dev. and Y=0 for web dev. 
G_attr = pd.read_csv(datadir + "GitHubDevelopers/musae_git_target.csv")
_ml = dict(zip(G_attr.id, G_attr.ml_target))
G.vs["Y"] = [_ml[i] for i in G.vs["id"]]


In [ ]:
## read feature 1793, set to 1 if present, 0 else
## this will be the "protected" attribute "X", which is correlated with "Y"
with open(datadir + "GitHubDevelopers/musae_git_features.json") as fp:
    features = json.load(fp)
G.vs['X'] = [int(1793 in features[str(i)]) for i in G.vs['id']]
print(np.corrcoef(G.vs['X'], G.vs['Y'])[0][1])


In [ ]:
## build degree features from each node's neighbours
for v in range(G.vcount()):
    x = np.array([G.vs[i]['Y'] for i in G.neighbors(v)])
    G.vs[v]['D_web'] = sum(x==0)
    G.vs[v]['D_ml'] = sum(x==1)


In [ ]:
## EDA - averages w.r.t. protected attribute
data = pd.DataFrame({'D_web':[i for i in G.vs['D_web']], 'D_ml':[i for i in G.vs['D_ml']], 'X':G.vs['X'], 'Y':G.vs['Y']})
data.groupby(by='X').mean()

## Build logistic regression models 

* **m1**: including the protected attribute X
* **m2**: excluding the protected attribute X

For the degree feature, we transform all values from $d$ to $\log(1+d)$.


In [ ]:
## model (m1) with protected variable X - single threshold

## first re-build data, taking 1+log for the degree-based features
data = pd.DataFrame({'X_web':[np.log(1+i) for i in G.vs['D_web']], \
                     'X_ml':[np.log(1+i) for i in G.vs['D_ml']],  \
                     'X':G.vs['X'], 'Y':G.vs['Y']})

## build model (m1)
X_m1 = sm.add_constant(data[['X_web', 'X_ml','X']])
model_m1 = sm.GLM(data['Y'], X_m1, family=sm.families.Binomial(link=families.links.Logit()))
results_m1 = model_m1.fit()

## model (m2) without protected variable X - single threshold

## build model (m2)
X_m2 = sm.add_constant(data[['X_web', 'X_ml']])
model_m2 = sm.GLM(data['Y'], X_m2, family=sm.families.Binomial(link=families.links.Logit()))
results_m2 = model_m2.fit()


## Maximize accuracy w.r.t. single threshold for both models

We see that even if we drop the protected variable X (model m2), i.e. "fairness through unawareness", there is still unfairness as indicated by the DI and SP scores, albeit slightly lower.


In [ ]:
## function to compute ACCuracy, DI and SP scores
## data: contains at least 'X' (protected attribute) and 'Y' (response)
## pred: predictions for the response
def eval_pred(data, pred):
    ACC = np.mean(data['Y'] == pred)
    DI = np.abs(np.mean(pred[data['X']==1]) - np.mean(pred[data['X']==0]))
    SP = np.sum([np.abs( np.mean( pred[(data['Y']==1-i) & (data['X']==1)]==i ) - \
                         np.mean( pred[(data['Y']==1-i) & (data['X']==0)]==i ) ) for i in [0,1] ])/2
    return ACC, DI, SP


In [ ]:
## threshold with best accuracy
t = np.arange(0,1,.01)
accs = [np.mean(data['Y'] == (results_m1.predict() >= i)) for i in t]
idx = np.argmax(accs)
pred = (results_m1.predict() >= t[idx])

## print resulting stats
acc, di, sp = eval_pred(data, pred)
print('\033[1m'+'model m1:','\033[0m'+f'accuracy: {acc:.5f}',f'DI: {di:.5f}',f'SP: {sp:.5f}',f'tau: {t[idx]:.2f}')

## threshold with best accuracy
accs = [np.mean(data['Y'] == (results_m2.predict() >= i)) for i in t]
idx = np.argmax(accs)
pred = (results_m2.predict() > t[idx])

## print resulting stats
acc, di, sp = eval_pred(data, pred)
print('\033[1m'+'model m2:','\033[0m'+f'accuracy: {acc:.5f}',f'DI: {di:.5f}',f'SP: {sp:.5f}',f'tau: {t[idx]:.2f}')


## Now with separate thresholds using model m1

Below is one-time work, we save (accuracy, DI, SP) for each value on a 100x100 grid covering thresholds $(\tau_0, \tau_1)$ for the two possible values of the protected variable.

Next we can query/print those results as needed, setting upper bounds for the two measures DI and SP.


In [ ]:
%%time

## one-time work - save arrays for further queries ( < 1 min on MAC Pro 2022 )
M_acc = np.zeros(shape=(100,100))
M_di = np.zeros(shape=(100,100))
M_sp = np.zeros(shape=(100,100))
_pred = results_m1.predict() ## save the predicted probabilities from model m1

## 100x100 grid
for i in np.arange(0,100):
    for j in np.arange(0,100):
        tau = [i/100,j/100] ## the two thresholds
        pred = np.array([v >= (tau[1] if c == 1 else tau[0]) for v, c in zip(_pred, data['X'])])
        M_acc[i,j],M_di[i,j],M_sp[i,j] = eval_pred(data, pred)


### query w.r.t. a few pairs of bounds


In [ ]:
## function to query w.r.t. a pair of thresholds
def query_results(M_acc, M_di, M_sp, th_di, th_sp):
    mask = ((M_di < th_di) & (M_sp < th_sp))
    _x, _y = np.where(mask)
    acc = M_acc[mask]
    idx = np.argmax(acc)
    return (_x[idx]/100, _y[idx]/100, acc[idx], M_di[mask][idx], M_sp[mask][idx])

## printing a few results
_bounds = [(1,1),(.65,1),(.05,1),(1,.48),(1,.05),(.05,.05)]
for (i,j) in _bounds:
    t0,t1,acc,di,sp = query_results(M_acc, M_di, M_sp, th_di=i, th_sp=j)
    print('\033[1m'+f'DI<={i:.2f} and SP<={j:.2f}:',
          '\033[0m'+f' tau0: {t0:.2f}',f' tau1: {t1:.2f}',
          f' accuracy: {acc:.5f}',f' DI: {di:.5f}',f' SP: {sp:.5f}')


### plotting all grid for: accuracy, DI and SP respectively

We see very similar pattern for the DI and SP scores.


In [ ]:
ax = sns.heatmap(M_acc, cmap='gray_r')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,100,4),np.arange(0,1,.04))
ax.set_yticks(np.arange(0,100,4),np.arange(0,1,.04))
plt.title('Accuracy', fontsize=20)
plt.ylabel(r'$\tau_0$', fontsize=16)
plt.xlabel(r'$\tau_1$', fontsize=16)
plt.show()
#plt.savefig('github_acc.png')
plt.clf()


In [ ]:
ax = sns.heatmap(M_di, cmap='gray')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,100,4),np.arange(0,1,.04))
ax.set_yticks(np.arange(0,100,4),np.arange(0,1,.04))
plt.title('DI', fontsize=20)
plt.ylabel(r'$\tau_0$', fontsize=16)
plt.xlabel(r'$\tau_1$', fontsize=16)
plt.show()
#plt.savefig('github_di.png')
plt.clf()


In [ ]:
ax = sns.heatmap(M_sp, cmap='gray')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,100,4),np.arange(0,1,.04))
ax.set_yticks(np.arange(0,100,4),np.arange(0,1,.04))
plt.title('SP', fontsize=20)
plt.ylabel(r'$\tau_0$', fontsize=16)
plt.xlabel(r'$\tau_1$', fontsize=16)
plt.show()
#plt.savefig('github_sp.png')
plt.clf()


# (2) ABCD graph - predicting large community membership

We generate an ABCD graph with one large community (size 1,000) and 10 small ones (size 100)

Variables:
* **Y** = "is a member of the large community", the target variable 
* **D** = node degree; the degree range is [30, 130)
* **X** = Protected variable (binary) ; engineered to be negatively correlated with degree

The prediction model is a simple threshold $\tau$ on thre dergees D, and we optimize w.r.t. accuracy. We demonstrate the existence of unfairness w.r.t. X when predicting Y. 

The unfairness can be lowered using two different thresholds $(\tau_0,\tau_1)$ for D depending on the value of X


In [ ]:
## Build ABCD graph, return igraph 'Graph' object

## for reproducibility
seed = 123
np.random.seed(seed)
random.seed(seed)

## parameters - community sizes and degree sequence
large_comm = 1000
small_comm = 100
n_small_comms = 10
c = np.concatenate( (np.array([large_comm]),np.repeat(small_comm,n_small_comms)) )
n_total = large_comm + (n_small_comms * small_comm)
min_deg = 30
degs = np.repeat(np.arange(min_deg,min_deg+100),20)

## generate graph
params = ABCDParams(degree_sequence=degs, community_size_sequence=c, vcount=n_total, xi=.1, num_outliers=0)
g = ABCDGraph(params).build()
G = g.exporter.to_igraph()

## build dataframe
df = pd.DataFrame()
df['Y'] = (np.array(G.vs['ground_truth_community'])==0)
df['D'] = G.degree()

## add negatively correlated attribute X, i.e. lower degrees when True
nodes = np.argsort(G.degree()) ## node from low to high degree
proba = np.linspace(.75,.25,n_total)
attribute = np.random.random(n_total) < proba
G.vs[nodes]['attr']  = attribute
df['X'] = G.vs['attr']
df.head()


In [ ]:
## correlations
df.corr()

## Maximize accuracy w.r.t. single threshold


In [ ]:
## single threshold
t = np.arange(min_deg,min_deg+100)
accs = [np.mean(df['Y'] == (df['D'] >= i)) for i in t]
idx = np.argmax(accs)
pred = (df['D'] >= t[idx])
acc, di, sp = eval_pred(df, pred)
print(f'accuracy: {acc:.3f}',f'DI: {di:.5f}',f'SP: {sp:.5f}','tau:',t[idx])


## Now using two thresholds


In [ ]:
%%time

## one-time work - save for further queries ( < 7 sec on MAC Pro 2022 )
M_acc = np.zeros(shape=(100,100))
M_di = np.zeros(shape=(100,100))
M_sp = np.zeros(shape=(100,100))

## 100x100 grid
for i in np.arange(0,100):
    for j in np.arange(0,100):
        tau = [i+min_deg,j+min_deg] ## the two thresholds
        pred = np.array([v >= (tau[1] if c == 1 else tau[0]) for v, c in zip(df['D'], df['X'])])
        M_acc[i,j],M_di[i,j],M_sp[i,j] = eval_pred(df, pred)


### query w.r.t. a few pairs of bounds


In [ ]:
_bounds = [(1,1),(.05,1),(1,.05),(.05,.05)]
for (i,j) in _bounds:
    t0,t1,acc,di,sp = query_results(M_acc, M_di, M_sp, th_di=i, th_sp=j)
    tau = ( int(100*t0+min_deg), int(100*t1+min_deg) )
    print('\033[1m'+f'DI<={i:.2f} and SP<={j:.2f}:',
          '\033[0m'+f' accuracy: {acc:.5f}',f' DI: {di:.5f}',f' SP: {sp:.5f}','tau:',tau)


### plotting all grid for: accuracy, DI and SP respectively


In [ ]:
ax = sns.heatmap(M_acc, cmap='gray_r')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,100,4),np.arange(min_deg,min_deg+100,4))
ax.set_yticks(np.arange(0,100,4),np.arange(min_deg,min_deg+100,4))
plt.title('Accuracy', fontsize=20)
plt.ylabel(r'$\tau_0$', fontsize=16)
plt.xlabel(r'$\tau_1$', fontsize=16)
plt.show()
#plt.savefig('abcd_acc.png')
plt.clf()


In [ ]:
ax = sns.heatmap(M_di, cmap='gray')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,100,4),np.arange(min_deg,min_deg+100,4))
ax.set_yticks(np.arange(0,100,4),np.arange(min_deg,min_deg+100,4))
plt.title('DI', fontsize=20)
plt.ylabel(r'$\tau_0$', fontsize=16)
plt.xlabel(r'$\tau_1$', fontsize=16)
plt.show()
#plt.savefig('abcd_di.png')
plt.clf()


In [ ]:
ax = sns.heatmap(M_sp, cmap='gray')
ax.invert_yaxis()
ax.set_xticks(np.arange(0,100,4),np.arange(min_deg,min_deg+100,4))
ax.set_yticks(np.arange(0,100,4),np.arange(min_deg,min_deg+100,4))
plt.title('SP', fontsize=20)
plt.ylabel(r'$\tau_0$', fontsize=16)
plt.xlabel(r'$\tau_1$', fontsize=16)
plt.show()
#plt.savefig('abcd_sp.png')
plt.clf()
